In [2]:
import pandas as pd
import numpy as np

import json
import gzip
import zipfile  

In [3]:
with zipfile.ZipFile("/dlabdata1/lugeon/AlexaRankByCategories.json.zip", "r") as z:
    for filename in z.namelist():  
        with z.open(filename) as f:  
            data = f.read()  
            jsondata = json.loads(data)  

In [17]:
root = jsondata['topsites']['category']['Top']

In [18]:
categories = list(root.keys())

In [19]:
categories

['Adult',
 'Arts',
 'Business',
 'Computers',
 'Games',
 'Health',
 'Home',
 'Kids_and_Teens',
 'News',
 'Recreation',
 'Reference',
 'Regional',
 'Science',
 'Shopping',
 'Society',
 'Sports',
 'World',
 '_Most_popular_',
 'Ar',
 'Gam',
 'R']

## Flatten everything

In [6]:
df = pd.DataFrame()

In [7]:
import collections

def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [8]:
root_f = flatten(root)

<ipython-input-7-72a859df90ce>:7: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  if isinstance(v, collections.MutableMapping):


In [9]:
url = []
cat = []
for k, l in root_f.items():
    url += l
    cat += [k] * len(l)
        

In [10]:
df['url'] = pd.Series(url)
df['cat0'] = pd.Series(cat)

In [11]:
df.drop_duplicates(subset='url', inplace=True)
df.reset_index(inplace=True, drop=True)

In [12]:
df['main_cat'] = df.apply(lambda row: row.cat0.split('_')[0], axis=1) 

In [13]:
df.drop('cat0', inplace=True, axis=1)

In [14]:
df['url'] = df.url.apply(lambda u : 'http://' + u) 

In [15]:
df.head()

,url,main_cat
0,http://cutepet.org,Adult
1,http://jasonmeador.com,Adult
2,http://taremeparadise.com,Adult
3,http://jlist.com,Adult
4,http://jastusa.com,Adult


In [16]:
df.to_csv('/dlabdata1/lugeon/websites_alexa.gz', compression='gzip')

## Most popular only

In [20]:
url = []
cat = []
subcat = []

for c in categories[:16]:
    print(c)
    for k in root[c].keys():
        if k == '_Most_popular_':
            continue
        print("  ", k)
        if type(root[c][k]) == list:
            l = root[c][k]
        else:
            l = root[c][k]['_Most_popular_']
        url += l
        cat += [c] * len(l)
        subcat += [k] * len(l)

Adult
   Arts
   Business
   Computers
   Death_and_Gore
   Games
   Image_Galleries
   News
   Recreation
   Regional
   Shopping
   Society
Arts
   Animation
   Architecture
   Art_History
   Awards
   Bodyart
   Chats_and_Forums
   Classical_Studies
   Comics
   Contests
   Costumes
   Crafts
   Cultures_and_Groups
   Design
   Digital
   Directories
   Education
   Entertainment
   Events
   Genres
   Graphic_Design
   Humanities
   Illustration
   Literature
   Magazines_and_E-zines
   Movies
   Music
   News_and_Media
   Online_Writing
   Organizations
   People
   Performing_Arts
   Periods_and_Movements
   Personal_Pages
   Photography
   Radio
   Regional
   Television
   Video
   Visual_Arts
   Web_Rings
   Weblogs
   Writers_Resources
Business
   Accounting
   Aerospace_and_Defense
   Agriculture_and_Forestry
   Arts_and_Entertainment
   Associations
   Automotive
   Biotechnology_and_Pharmaceuticals
   Business_and_Society
   Business_Services
   Chemicals
   Construction_a

In [21]:
df = pd.DataFrame({'url': url, 'cat0': cat, 'subcat0': subcat})

In [22]:
df['url'] = df.url.apply(lambda u : 'http://www.' + u) 

In [23]:
df

,url,cat0,subcat0
0,http://www.nhentai.net,Adult,Arts
1,http://www.furaffinity.net,Adult,Arts
2,http://www.literotica.com,Adult,Arts
3,http://www.iafd.com,Adult,Arts
4,http://www.sofurry.com,Adult,Arts
...,...,...,...
20056,http://www.nays.org,Sports,Youth_and_High_School
20057,http://www.nhsca.com,Sports,Youth_and_High_School
20058,http://www.naysi.com,Sports,Youth_and_High_School
20059,http://www.chap.com,Sports,Youth_and_High_School


In [37]:
df.to_csv('/dlabdata1/lugeon/websites_alexa_mostpop2.gz', compression='gzip')

In [26]:
df = pd.read_csv('/dlabdata1/lugeon/websites_alexa_mostpop2.gz', index_col=0)

In [27]:
df.head()

,url,cat0,subcat0
0,http://www.nhentai.net,Adult,Arts
1,http://www.furaffinity.net,Adult,Arts
2,http://www.literotica.com,Adult,Arts
3,http://www.iafd.com,Adult,Arts
4,http://www.sofurry.com,Adult,Arts


In [28]:
count = df.cat0.value_counts()
count

Sports            2969
Business          2221
Computers         1863
Shopping          1664
Arts              1531
Society           1480
Recreation        1445
Health            1407
Science           1130
Games              805
Reference          712
News               703
Home               596
Kids_and_Teens     567
Regional           500
Adult              468
Name: cat0, dtype: int64

In [7]:
biggest_cat = count[count >= 1000].index.values
biggest_cat

array(['Sports', 'Business', 'Computers', 'Shopping', 'Arts', 'Society',
       'Recreation', 'Health', 'Science'], dtype=object)

In [8]:
df_red = df[df.apply(lambda row: row.cat0 in biggest_cat, axis=1)]

In [9]:
df_red

,url,cat0,subcat0
468,http://www.sankakucomplex.com,Arts,Animation
469,http://www.anime-planet.com,Arts,Animation
470,http://www.pokemon.com,Arts,Animation
471,http://www.viz.com,Arts,Animation
472,http://www.animefreak.tv,Arts,Animation
...,...,...,...
20056,http://www.nays.org,Sports,Youth_and_High_School
20057,http://www.nhsca.com,Sports,Youth_and_High_School
20058,http://www.naysi.com,Sports,Youth_and_High_School
20059,http://www.chap.com,Sports,Youth_and_High_School


In [29]:
df_train = pd.DataFrame()
df_test = pd.DataFrame()

for cat in df.cat0.unique():
    df_cat = df[df.cat0 == cat]
    
    tot_samples = df_cat.shape[0]
    frac = 0.1
    sep = int(tot_samples * frac)
    
    ix = np.arange(tot_samples)
    np.random.shuffle(ix)
    
    df_train = pd.concat((df_train, df_cat.iloc[ix[sep:]]))
    df_test = pd.concat((df_test, df_cat.iloc[ix[:sep]]))
    
    
    

In [30]:
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [31]:
df_train

,url,cat0,subcat0
0,http://www.faapy.com,Adult,Image_Galleries
1,http://www.caballerovip.com,Adult,Business
2,http://www.swingular.com,Adult,Society
3,http://www.persiankitty.com,Adult,Computers
4,http://www.dollforum.com,Adult,Recreation
...,...,...,...
18056,http://www.tulliallanbowlingclub.50megs.com,Sports,Bowling
18057,http://www.polevaultpower.com,Sports,Track_and_Field
18058,http://www.columbustabletennisclub.com,Sports,Table_Tennis
18059,http://www.ngthomas.co.uk,Sports,Software


In [32]:
df_test

,url,cat0,subcat0
0,http://www.xcitement.com,Adult,News
1,http://www.tonybatman.com,Adult,News
2,http://www.amibearornot.com,Adult,Recreation
3,http://www.legaction.com,Adult,News
4,http://www.sextvx.com,Adult,Image_Galleries
...,...,...,...
1995,http://www.slamonline.com,Sports,Basketball
1996,http://www.collegiatedirectories.com,Sports,College_and_University
1997,http://www.sbabadminton.org,Sports,Badminton
1998,http://www.lazer-zone.net,Sports,Laser_Games


In [33]:
df_train.to_csv('/dlabdata1/lugeon/websites_alexa_mp2_full_train.gz', compression='gzip')
df_test.to_csv('/dlabdata1/lugeon/websites_alexa_mp2_full_valid.gz', compression='gzip')

### Taking only a subsample from each categories

In [45]:
df = pd.read_csv('/dlabdata1/lugeon/websites_alexa.gz', index_col=0)
df

/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,url,main_cat
0,http://cutepet.org,Adult
1,http://jasonmeador.com,Adult
2,http://taremeparadise.com,Adult
3,http://jlist.com,Adult
4,http://jastusa.com,Adult
...,...,...
1077657,http://cafef.vn,World
1077658,http://thanhnien.vn,World
1077659,http://lazada.vn,World
1077660,http://soha.vn,World


In [46]:
df = df[df.main_cat != 'World']

In [47]:
df.main_cat.value_counts()

Regional      316246
Business      118663
Society        71675
Recreation     44638
Shopping       42487
Arts           39362
Computers      32995
Sports         30515
Health         24037
Science        23580
Reference      22682
Games           8900
Home            5776
Kids            4412
Adult           4200
News            3880
Name: main_cat, dtype: int64

In [50]:
nb_max_samples = 20_000

df_train = pd.DataFrame()
df_test = pd.DataFrame()

for cat in df.main_cat.unique():
    df_cat = df[df.main_cat == cat]
    
    tot_samples = min(df_cat.shape[0], nb_max_samples)
    frac = 0.2
    sep = int(tot_samples * frac)
    
    ix = np.arange(tot_samples)
    np.random.shuffle(ix)
    
    df_train = pd.concat((df_train, df_cat.iloc[ix[sep:]]))
    df_test = pd.concat((df_test, df_cat.iloc[ix[:sep]]))
    
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [55]:
df_test.main_cat.value_counts()

Sports        4000
Health        4000
Regional      4000
Business      4000
Reference     4000
Computers     4000
Arts          4000
Shopping      4000
Society       4000
Science       4000
Recreation    4000
Games         1780
Home          1155
Kids           882
Adult          840
News           776
Name: main_cat, dtype: int64

In [53]:
df_train.to_csv('/dlabdata1/lugeon/websites_alexa_20_000_train.gz', compression='gzip')
df_test.to_csv('/dlabdata1/lugeon/websites_alexa_20_000_valid.gz', compression='gzip')